In [4]:
# !pip install ensemble-boxes
from ensemble_boxes import weighted_boxes_fusion
import numpy as np
import pandas as pd
from collections import defaultdict
from pycocotools.coco import COCO

test_json = '/opt/ml/dataset/test.json'
test_coco = COCO(test_json)


# step 1: collate all csv files
submission_files = [
    '/opt/ml/level2_objectdetection_cv-level2-cv-15/mmdetection/submission_hj_1.csv',
    '/opt/ml/level2_objectdetection_cv-level2-cv-15/mmdetection/submission_sy_1.csv',
]

# # # # # # # # # # # # # # # config -> 조절
skip_box_thr = 0.1
iou_thr = 0.5
# # # # # # # # # # # # # # #

image_classes = defaultdict(list)
image_confidence = defaultdict(list)
image_boxes = defaultdict(list)
img_size = 1024

for submission_file in submission_files:
    df = pd.read_csv(submission_file)
    cnt = 1
    for img_id, prediction_string in zip(df['image_id'], df['PredictionString']):
        ps = list(prediction_string.split())
        num_boxes = len(ps)//6
        for i in range(num_boxes):
            cls = int(ps[6*i])
            conf, x1, y1, x2, y2 = map(float,ps[6*i+1:6*(i+1)])
            
            image_classes[img_id].append(cls)
            image_confidence[img_id].append(conf)
            image_boxes[img_id].append([round(x1/img_size,5),round(y1/img_size,5),
                round(x2/img_size,5),round(y2/img_size,5)])

prediction_strings = []
file_names = []

image_ids = pd.read_csv(submission_files[0])['image_id'].tolist()
for img_id in image_ids:
    prediction_string = ''
    # print(img_id)
    boxes_list = image_boxes[img_id]
    scores_list = image_confidence[img_id]
    labels_list = image_classes[img_id]

    boxes, scores, labels = weighted_boxes_fusion([boxes_list], [scores_list], [labels_list], iou_thr=iou_thr ,skip_box_thr=skip_box_thr)
    for box, score, label in zip(boxes, scores, labels):
        prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * img_size) + ' ' + str(box[1] * img_size) + ' ' + str(box[2] * img_size) + ' ' + str(box[3] * img_size) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(img_id)

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('/opt/ml/level2_objectdetection_cv-level2-cv-15/mmdetection/ensemble.csv', index=None)
submission.head()



loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


,PredictionString,image_id
0,7 0.99960857629776 216.52471923828125 49.82777...,test/0000.jpg
1,6 0.9772614240646362 763.556640625 661.4655151...,test/0001.jpg
2,1 0.934711217880249 424.805419921875 268.65066...,test/0002.jpg
3,9 0.9991870522499084 157.36749267578125 263.79...,test/0003.jpg
4,0 0.7142089009284973 422.87786865234375 408.98...,test/0004.jpg
